In [2]:
import pandas as pd
import plotly.express as px
from datetime import datetime
from zoneinfo import ZoneInfo
import re

# -------------------------
# LOAD DATA
# -------------------------
df = pd.read_csv("Play Store Data.csv")

# -------------------------
# CLEAN COLUMNS
# -------------------------

# installs
df["Installs"] = (
    df["Installs"].astype(str)
    .str.replace("+","")
    .str.replace(",","")
)
df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

# rating & reviews
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
df["Reviews"] = pd.to_numeric(df["Reviews"], errors="coerce")

# size → MB
def size_to_mb(x):
    if isinstance(x,str) and "M" in x:
        return float(x.replace("M",""))
    return None

df["Size_MB"] = df["Size"].apply(size_to_mb)

# date
df["Last Updated"] = pd.to_datetime(df["Last Updated"], errors="coerce")
df["Month"] = df["Last Updated"].dt.to_period("M").astype(str)

# -------------------------
# FILTERS
# -------------------------

# app name without numbers
no_number_name = ~df["App"].astype(str).str.contains(r"\d")

filtered = df[
    (df["Rating"] >= 4.2) &
    (no_number_name) &
    (df["Category"].str.startswith(("T","P"), na=False)) &
    (df["Reviews"] > 1000) &
    (df["Size_MB"].between(20,80))
]

# -------------------------
# MONTHLY INSTALLS BY CATEGORY
# -------------------------
monthly = (
    filtered.groupby(["Month","Category"])["Installs"]
    .sum()
    .reset_index()
    .sort_values("Month")
)

# cumulative installs
monthly["Cumulative"] = (
    monthly.groupby("Category")["Installs"]
    .cumsum()
)

# -------------------------
# HIGHLIGHT >25% MoM GROWTH
# -------------------------
monthly["MoM_Growth"] = (
    monthly.groupby("Category")["Installs"]
    .pct_change()
)

monthly["Highlight"] = monthly["MoM_Growth"] > 0.25

# increase intensity marker
monthly["Intensity"] = monthly["Highlight"].map({True: 1.0, False: 0.6})

# -------------------------
# LEGEND TRANSLATIONS
# -------------------------
legend_map = {
    "Travel & Local": "Voyage et Local",
    "Productivity": "Productividad",
    "Photography": "写真"
}

monthly["Category_Label"] = monthly["Category"].replace(legend_map)

# -------------------------
# TIME CHECK — 4PM–6PM IST
# -------------------------
now_ist = datetime.now(ZoneInfo("Asia/Kolkata"))

if 16 <= now_ist.hour < 18:

    # -------------------------
    # STACKED AREA CHART
    # -------------------------
    fig = px.area(
        monthly,
        x="Month",
        y="Cumulative",
        color="Category_Label",
        title="Cumulative Installs Over Time by Category",
        line_group="Category_Label"
    )

    fig.show()

else:
    print("Chart hidden — visible only between 4 PM and 6 PM IST")


Chart hidden — visible only between 4 PM and 6 PM IST
